<center><h1 style="color:red">Social Distancing Detector</h1><br><center>

In [3]:
import cv2 as cv
import numpy as np
import imutils
from scipy.spatial import distance

In [4]:
#the following function is use to calculate distance between 2 persons
#we have assume 120 euclidean distance as safe distance, you can change it as per your need

def calculate_distance(indexes):

    centres = []
    colors = {}
    dict = {}
    Safe_Distance = 120

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        x_mid = int(x+w/2)
        y_mid = int(y+h/2)
        dict[i] = 0
        centres.append([i,x_mid,y_mid])

    for i in range(len(indexes)-1):
        for j in range(i+1,len(indexes)):
            dist = distance.euclidean(centres[i][1:3],centres[j][1:3])

            if dist<Safe_Distance:
                dict[centres[i][0]]=1
                dict[centres[j][0]]=1
    risky=0
    for i,val in dict.items():
        if val==0:
            colors[i] = (0,255,0)

        else:
            colors[i] = (0,0,255)
            risky+=1

    return colors,risky


In [5]:
# we would be using yolov3 to detect person in the images 

net = cv.dnn.readNet('yolov3.weights','yolov3.cfg')
classes = []

with open('coco.names','r') as f:
    classes = f.read().split("\n")

cap = cv.VideoCapture("street.mp4")

In [6]:
while True:
    frame,img = cap.read()

    if frame:

        img = imutils.resize(img, width=1280,height=1000)
        height,width,_ = img.shape

        blob = cv.dnn.blobFromImage(img,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

        net.setInput(blob)

        output_layers_names = net.getUnconnectedOutLayersNames()
        layerOutputs = net.forward(output_layers_names)

        boxes = []
        confidences = []
        class_ids = []

        for output in layerOutputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5 and class_id == 0:
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    x = int(center_x-w/2)
                    y = int(center_y-h/2)

                    boxes.append([x,y,w,h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)


        indexes = cv.dnn.NMSBoxes(boxes,confidences,0.5,0.4)

        font = cv.FONT_HERSHEY_PLAIN

        color = (0,255,0)
        risky=0

        if(len(indexes)>1):
            colors,risky = calculate_distance(indexes)

        if len(indexes)>0:

            for i in indexes.flatten():
                x,y,w,h = boxes[i]
                label = str(classes[class_ids[i]])

                if len(indexes)>1:
                    color = colors[i]

                if color==(0,255,0):
                    label='Safe'
                    k=55

                else:
                    label="Risk!!"
                    k=62

                if(risky>0):
                    cv.rectangle(img, (0, 0), (462, 30), (255, 255, 255), -1)
                    cv.putText(img,"NO OF SOCIAL DISTANCE VIOLATIONS :"+f"{risky}",(10,20),font,1.4,(0,0,255),2)
                else:
                    cv.rectangle(img, (0, 0), (150, 30), (255, 255, 255), -1)
                    cv.putText(img, "  SAFE  ", (15, 20), font, 1.4, (0,255,0), 2)

                cv.rectangle(img,(x,y),(x+w,y+h),color,2)
                cv.rectangle(img,(x,y-5),(x+k,y+15),(255,255,255),-1)
                cv.putText(img,label,(x+5,y+10),font,1.3,color,2)
        
        cv.imshow('img',img)
        key = cv.waitKey(1)

        if key == ord('q'):
            break

    else:
        break


In [7]:
cap.release()
cv.destroyAllWindows()

<center><h1 style='color:red;font-family:cursive'>Thank You !!</h1><center>